# Assignment 7

Name 1: <br/>
Student id 1: <br/>
Email 1: <br/>


Name 2: <br/>
Student id 2: <br/>
Email 2: <br/> 

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the Python files for exercises 1 and 2. There is no need to submit the data files. <br/>
Upload the zipped folder in Teams. Make sure to click on "Turn-in" after your upload your submission, otherwise the assignment will not be considered as submitted. Only one from the group should make the submisssion.

---

## Exercise 1: Count-Trees (5 points)

Your task is to implement a count-tree with variable maximum history. It is a memory-efficient way to store n-gram counts which can also be used to create an intuitive back-off after the tree is pruned.

The tree object should support the following four operations:
- Increment a count of a specific n-gram (even if not present)
- Retrieve counts given the history (variable length)
- Retrieve the conditional probability of a word given the history. (Proportion count between branches)
- Pruning all nodes with counts less or equal to $k$

**1.1 (2 points)**

Make sure your implementation is correct by passing the asserts in the first code cell.

**1.2 (1 point)**

The next cell will incrementally add a quad-gram to the tree. Plot the perplexity of trigram language model (induced by this count tree) against the number of added n-grams. Comment on the curve shape. Smooth this language model with a zerogram distribution using a linear combination ($0.75\times p_4 + 0.25\times p_0$).

**1.3 (1 point)**

For the given range of thresholds, prune your tree and see how the threshold affects the performance. Plot the results (perplexity vs. threshold).

**1.4 (1 point)**

1. If you first prune with threshold $k_1$ and get tree $t_1$, then prune with $k_2$ and get $t_2$ what will be the relationship between $t_1$ and $t_2$ if $k_1 \ge k_2$? (0.25 points)
2. What is the memory benefit of count trees, in comparison to storing the counts as a dictionary `{n-gram:freq}`? (0.25 points)
3. If we pruned the tree so that only the first level is preserved, what distribution could we model with this tree? (0.25 points)
4. Pruning the count tree is said to be a dynamic way of smoothing the language model. Elaborate on how this smoothing happens. (0.25 points)

In [ ]:
from importlib import reload
import exercise_1
exercise_1 = reload(exercise_1)

tree = exercise_1.CountTree(n=4)

assert tree.get("") == 0
tree.add("ABCE")
tree.add("ABCD")
tree.add("ABCD")
tree.add("QBCD")
tree.add("QQCD")
tree.add("BCDA")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("1234")
tree.add("5634")
assert tree.get("ABCD") == 2
assert tree.get("ABCX") == 0
assert tree.get("BCD") == 3
assert tree.get("D") == 4
assert tree.get("CD") == 4
assert tree.get("1234") == 5
assert tree.get("5634") == 1
tree.prune(4)
assert tree.get("ABCD") == 4
assert tree.get("XXCD") == 4
assert tree.get("D") == 4
assert tree.get("1234") == 5
assert tree.get("5634") == 1

In [ ]:
import matplotlib.pyplot as plt

plot_x = []
plot_y = []

ngrams = []
tree = exercise_1.CountTree(n=4)
with open("data/alice_in_wonderland.txt", "r") as f:
  tokens = f.read().lower().split()
  for i in range(len(tokens)-4):
    ngrams.append(tokens[i:i+4])

vocab = set(tokens)
for i,ngram in enumerate(ngrams):
  tree.add(ngram)
  if i % 1000 == 0:
    plot_x.append(i)
    plot_y.append(tree.perplexity(ngrams, vocab))

plt.plot(plot_x, plot_y)

In [ ]:
plot_x = []
plot_y = []

for threshold in [1,2,3,4,5,10,25,50,75,100]:
  tree.prune(threshold)
  plot_x.append(threshold)
  plot_y.append(tree.perplexity(ngrams, vocab))

print(plot_x)
print(plot_y)

plt.plot(plot_x, plot_y)

## Exercise 2: Kneser-Ney Smoothing (5 points)

This exercise aims to provide a basic understanding of Kneser-Ney Smoothing. Kneser-Ney Smoothing makes use of *continuation counts* of words for lower order n-grams, given as

\begin{equation}
C_{KN} = 
\begin{cases}
\text{count}(\bullet) & \text{for highest order} \\
\text{continuationcount}(\bullet) & \text{for lower orders}
\end{cases}
\end{equation}


For a trigram distribution, Kneser-Ney Smoothing is implemented using the following equations:

$$P_{KN}(w_3|w_1, w_2) = \frac{\max\{N(w_1 w_2 w_3)-d,0\}}{N(w_1 w_2)} + \lambda(w_1, w_2)P_{KN}(w_3|w_2)$$

$$P_{KN}(w_3|w_2) = \frac{\max\{N_{+}(\bullet w_2 w_3)-d,0\}}{N_{+}(\bullet w_2 \bullet)} + \lambda(w_2)P_{KN}(w_3)$$

\begin{equation}
P_{KN}(w_3) = \begin{cases}
\frac{N_{+}(\bullet w_3)}{N_{+}(\bullet \bullet)} & \text{if $w_3 \in$ V} \\
\frac{1}{V} & \text{otherwise}
\end{cases}
\end{equation}

$\lambda$ is used to normalise the discounted probability mass and is given by

$$\lambda(w_1, w_2) = \frac{d}{N(w_1 w_2)} \cdot N_{+}(w_1 w_2 \bullet)$$

$$\lambda(w_2) = \frac{d}{N(w_2)} \cdot N_{+}(w_2 \bullet)$$

**2.1 (4.5 points)**

* Your first task is to understand what these terms represent and fill it in the table below (4-5 words each).

* Create a trigram-level model on the given text, `alice_in_wonderland.txt`. Write your implementation in the file `exercise_2.py`. Preprocess the text by punctuation removal, lowercasing, and tokenisation. There is no need to split the data into train and test sets. (0.5 points)

In [ ]:
from importlib import reload
import exercise_2
exercise_2 = reload(exercise_2)

file = open("data/alice_in_wonderland.txt", "r")
text = file.read()

# TODO: Preprocess text
tokens = exercise_2.preprocess(text)

* 
Write a simple class `KneserNey` in `exercise_2` that calculates the different parameters required for finding the trigram conditional probability. You may modify the function signature and add other functionality as required. <br/>
Now, consider the trigrams `"alice said nothing"` and `"alice said nichts"`. For these trigrams, estimate the values mentioned in the table given below and fill in the obtained results. The discounting parameter *d* = 0.75. (3 points)

In [ ]:
KN_model = exercise_2.KneserNey(tokens, d=0.75, N=3)

t1 = "alice said nothing"
t2 = "alice said nichts"

# TODO
# Get the required parameters
KN_model.get_params(t1)
KN_model.get_params(t2)

| Term in Kneser-Ney |   Value t1  | Value t2 | Description | 
|---|---|---| --- |
|$N(w_1w_2w_3)$|  |  | |
|$N(w_1 w_2)$|  |  | |
|$N_{+}( \bullet w_2 w_3)$|  |  | |
|$N_{+}( \bullet w_2 \bullet)$|  |  | |
|$N_{+}( \bullet w_3)$|  |  | |
|$N_{+}( \bullet \bullet)$|  |  | |
|$N_{+}(w_1 w_2 \bullet)$| | | |
|$N_{+}(w_2 \bullet)$|  |  | |
|$\lambda(w_1 w_2)$| | | |
|$\lambda(w_2)$| | | |

* Using the values obtained above, manually calculate $P_{KN}(w_3)$, $P_{KN}(w_3|w_2)$, and $P_{KN}(w_3|w_1, w_2)$ for the given trigrams. (1 point)

**2.2 (0.5 points)**

Take a look at this [video](https://www.youtube.com/watch?v=cbAxvpBFyNU) on Kneser-Ney smoothing by Dan Jurafksy. Make sure to undestand his *San Francisco* example. <br/>
How will Kneser-Ney Smoothing handle the following bigrams (answer in 3-4 sentences)? 

* Abu Dhabi

* Game Over


## Bonus (2 points)

For each of the smoothing techniques below,

1. Laplace/add-1 smoothing (0.3 points)
2. Add-$\alpha$ smoothing (0.3 points)
3. Linear interpolation (0.3 points)
4. Absolute discounting (0.3 points)
5. Good-Turing (0.3 points)
6. Kneser-Ney smoothing (0.3 point)

* Give the intuition behind it
* State at least one drawback and
* Explain how the ensuing smoothing technique accounts for this drawback.

You can do so in continuous text or in bullet points. Write 3-5 sentences for each technique. For Kneser-Ney smoothing, you should suggest *and explain* an improved version from the literature, e.g. [here](http://nrs.harvard.edu/urn-3:HUL.InstRepos:25104739) (this tutorial may also be helpful for the rest of the exercise).

Please note that while the points for this bonus exercise are the immediate motivation, your self-made comparison will be highly beneficial for the exam.